In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate


In [ ]:
def file_opening(name):
    """
    Open a CSV file and extract scenario names and corresponding data.

    Parameters:
    name (str): Path to the CSV file.

    Returns:
    tuple:
        - scenarios (list): List of scenario names (column headers excluding the first one).
        - data (list of arrays): List of arrays containing data for each scenario.
    """
    # Read the CSV file
    df = pd.read_csv(name, sep=';')

    # Extract scenario names
    scenarios = df.columns[1:].tolist()

    # Extract corresponding data as a list of arrays
    data = [df.iloc[:, k+1].values for k in range(len(scenarios))]

    return scenarios, data


In [ ]:
def file_opening_dict(file_name):
    """
    Open a CSV file and return its contents as a dictionary.

    Parameters:
    file_name (str): Path to the CSV file.

    Returns:
    dict: A dictionary where keys are column names and values are lists of column data.
    """
    # Read the CSV file with the first column as index
    df = pd.read_csv(file_name, index_col=0, sep=';')

    # Convert dataframe to dictionary with lists as values
    scenarios = df.to_dict(orient='list')

    return scenarios


In [ ]:
### Physical parameters ###
g=9.81
rho_air=1.2
fuel=32.3
R_bat=0.35
B_lim=1.3
Na_max=550 #rad/s

In [ ]:
def EC_calculation_th(body, engine, path, trans):
    """
    Compute energy consumption (EC) based on vehicle body, engine, path, and transmission parameters.

    Parameters:
    body (tuple): Vehicle body parameters.
    engine (tuple): Engine parameters.
    path (tuple): Path characteristics.
    trans (tuple): Transmission parameters.

    Returns:
    tuple:
        - EC (float): Total energy consumption.
        - breakdown (list): List of energy consumption components.
    """
    ### Extracting input parameters ###

    # Body parameters
    M_body, M_payload, r0, Cd, A, Iw, Rw, transf, transg, stop_start = body

    # Engine parameters
    P_max, ne, D, fmep0, p0, Q0, N_idle, cs = engine

    # Path parameters
    (J3p, K1p, K2p, rate_acc, J0p, H, w, urban, B, PaM, mu_N, M_cargo, Pacc, 
     payload_factor, M_eq, Cd_eq, cold_start_fact, dist, t_idle) = path

    # Transmission parameters
    ntr, a_tr, S, Ne = trans

    # Fuel properties
    LHV = fuel  # Assuming `fuel` is predefined

    ### Calculated Parameters ###

    M_tot = M_body + M_cargo + payload_factor * M_payload + M_eq
    Cd *= Cd_eq
    sigma_t = transf * transg / Rw
    conv = LHV * 10

    ### Computed Parameters for Integration Model ###

    K1 = K1p
    K2 = K2p
    ta = K1 / PaM
    da = K2 / PaM

    Nc = Ne * np.pi / 30 * mu_N
    N_idle = N_idle * np.pi / 30
    mu_a = PaM * M_tot / P_max
    Na = N_idle + Na_max * np.sqrt(mu_a)

    # Ensuring Na is not lower than Nc for realistic driving cycles
    if Na < Nc:
        Na = Nc

    ### Integrals calculation ###

    J1 = 1 - K1 / B
    J1_cruise = 1 - K1 / B - da
    J0_cruise = J0p * J1_cruise
    J3_cruise = J3p * J1_cruise
    J0 = J0p * (J1 - da * (1 - 1 / rate_acc))
    J3 = J3p * (J1 - da * (1 - rate_acc ** 2))

    Chi1 = urban * Nc * J0_cruise + (1 - urban) * sigma_t * J1_cruise + Na * ta
    Chi3 = urban * Nc ** 3 * J0_cruise + (1 - urban) * sigma_t ** 3 * J3_cruise + Na ** 3 * ta

    ### External Forces Energy Consumption Computation ###

    rolling = r0 * M_tot * g * J1 / conv / ne / ntr
    drag = 0.5 * rho_air * Cd * A * J3 / conv / ne / ntr
    inertia = (M_tot + 4 * Iw / Rw ** 2) * K1 / conv / ne / ntr
    grade = M_tot * g * H / conv / ne / ntr
    wind = 0.5 * rho_air * Cd * A * w ** 2 * J1 / conv / ne / ntr

    ### Powertrain Losses Energy Consumption Computation ###

    friction = fmep0 / (4 * np.pi) * D * Chi1 / conv / ne
    pumping = p0 / (4 * np.pi) * D * Chi3 / conv / ne
    thermal = Q0 * D * J0 / conv / ne
    cold_engine = cs * P_max * cold_start_fact / conv / dist
    accessories = Pacc * (J0 + t_idle) / conv / ne
    transmission = a_tr * P_max * Chi1 / conv / ntr / ne
    synchronization = urban * S / conv / ne
    idling = (fmep0 / (4 * np.pi) * D * N_idle * t_idle / conv / ne +
              Q0 * D * t_idle / conv / ne) * stop_start

    ### Final Energy Consumption Calculation ###

    EC = (rolling + drag + inertia + grade + wind + friction + pumping +
          thermal + accessories + transmission + synchronization + cold_engine + idling)

    return EC, [rolling, drag, inertia, grade, wind, friction, pumping, 
                thermal, accessories, transmission, synchronization, cold_engine, idling]


In [ ]:
def EC_calculation_elec(body, engine, path, trans):
    """
    Compute energy consumption (EC) for an electric vehicle based on body, engine, path, and transmission parameters.

    Parameters:
    body (tuple): Vehicle body parameters.
    engine (tuple): Engine parameters.
    path (tuple): Path characteristics.
    trans (tuple): Transmission parameters.

    Returns:
    tuple:
        - EC (float): Total energy consumption.
        - breakdown (list): List of energy consumption components.
    """
    ### Extracting input parameters ###

    # Body parameters
    (M_body, M_payload, r0, Cd, A, Iw, Rw, transf, P_bat, P_bat_min, U_bat, 
     P_charg, n_bat) = body

    # Engine parameters
    P_e, Tmax, ne, alpha, epsilon, betha = engine

    # Path parameters
    (J3p, K1p, K2p, rate_acc, J0p, H, w, urban, B, PaM, mu_N, M_cargo, Pacc, 
     payload_factor, M_eq, Cd_eq, dist, t_idle) = path

    # Transmission parameters
    a_tr, ntr = trans

    ### Calculated Parameters ###
    conv = 36  # Conversion factor
    M_tot = M_body + M_cargo + payload_factor * M_payload + M_eq
    Cd *= Cd_eq
    sigma_t = transf / Rw

    ### Regenerative Braking Efficiency Calculation ###
    if B > B_lim / 2:
        nregen = 1 - ((2 * B - B_lim) ** 2) / (4 * B * B)
    else:
        nregen = 1

    ### Computed Parameters for Integration Model ###
    K1 = K1p
    K2 = K2p
    ta = K1 / PaM
    da = K2 / PaM
    mu_a = PaM / (P_e / M_tot)
    P_a = P_e * mu_a
    C2 = Tmax * (mu_a * Tmax) * ta

    ### Integrals calculation ###
    J1 = 1 - K1 / B * (1 - rate_acc ** 2)
    J1a = J1 - da * (1 - rate_acc ** 2)
    J0 = J0p * (1 - (K1 / B - da) * (1 - 1 / rate_acc))
    J3 = J3p * (1 - K1 / B * (1 - rate_acc ** 2) - da * (1 - rate_acc ** 2))
    Chi1 = sigma_t

    ### External Forces Energy Consumption Computation ###
    rolling = r0 * M_tot * g / (conv * ne * ntr * n_bat)
    drag = 0.5 * rho_air * Cd * A * J3 / (conv * ne * ntr * n_bat)
    inertia = (M_tot + 4 * Iw / (Rw * Rw)) * K1 / (conv * ne * ntr) * (1 - nregen) / n_bat
    grade = M_tot * g * H / (conv * ne * ntr * n_bat)
    wind = 0.5 * rho_air * Cd * A * w ** 2 * J1 / (conv * ne * ntr * n_bat)

    ### Engine/Drivetrain Losses Energy Consumption Computation ###
    friction = alpha * Chi1 / (conv * ne * n_bat)
    copper = epsilon * C2 / (conv * ne * n_bat)
    converter = betha * J0 / (conv * ne * n_bat)
    accessories = Pacc * (J0p + t_idle) / (conv * ne * n_bat)
    transmission = a_tr * P_e * Chi1 / (conv * ntr * ne * n_bat)

    ### Battery Losses Energy Consumption Computation ###
    P2_cruise = (1 / J0) * ((rolling + drag + grade + wind + friction + copper + converter + accessories + transmission) * conv) ** 2
    P2_acc = P_a * K1 * M_tot
    P2 = P2_acc + P2_cruise
    battery = R_bat / U_bat / U_bat * P2 / (conv * n_bat)

    ### Final Energy Consumption Calculation ###
    EC = (rolling + drag + inertia + grade + wind + friction + copper + 
          converter + accessories + transmission + battery)

    return EC, [rolling, drag, inertia, grade, wind, friction, copper, 
                converter, accessories, transmission, battery]


In [ ]:
### Preparation of dictionnary containing the parameters of the case studies ###

test_list=file_opening('EC_preparation.csv')
body_dict=file_opening_dict('body_preparation.csv')
engine_dict=file_opening_dict('engine_preparation.csv')
trans_dict=file_opening_dict('trans_preparation.csv')
cycle_dict=file_opening_dict('DC_preparation.csv')

testEV_list=file_opening('EC_preparation_EV.csv')
bodyEV_dict=file_opening_dict('body_preparation_EV.csv')
engineEV_dict=file_opening_dict('engine_preparation_EV.csv')
transEV_dict=file_opening_dict('trans_preparation_EV.csv')
cycleEV_dict=file_opening_dict('DC_preparation_EV.csv')


In [ ]:
""" 
Energy Consumption are calculated based on the parameters collected for the different case studies. 
Results are compared with the empirical EC results realized by car manufacturer.
The table produced by this algorithm presents the results for GV.
It displays the car model, engine and driving cycle of the test, 
then compares the theoretical EC calculation with with Empirical Results, and measures the relative error of the model.

"""

EC_test = []
EC_reel = []
M_list = []
D_list = []

# Iterate through the test list and calculate EC values
for k in range(len(test_list[1])):
    try:
        body_name, cycle_name, engine_name, trans_name = test_list[1][k][:4]
        EC_reel.append(float(test_list[1][k][4]))  # Convert EC_reel values to float
        EC_test.append(EC_calculation_th(body_dict[body_name], engine_dict[engine_name], cycle_dict[cycle_name], trans_dict[trans_name])[0])
    except KeyError as e:
        print(f"Warning: Missing key {e} in one of the dictionaries. Skipping row {k}.")
    except ValueError:
        print(f"Warning: Invalid EC value in row {k}. Skipping.")

# Convert lists to NumPy arrays
EC_test = np.array(EC_test)
EC_reel = np.array(EC_reel, dtype=float)

# Prepare table for display
table = [
    ['Car', 'Cycle', "Th. EC[l/100km]", "Emp. EC[l/100km]", "Error", "Rel.Error [%]"],
    *[
        [
            test_list[1][i][0],  # Car name
            test_list[1][i][1],  # Cycle name
            round(EC_test[i], 2), # Theoretical EC
            round(EC_reel[i], 2), # Empirical EC
            round(EC_test[i] - EC_reel[i], 2), # Absolute error
            round((EC_test[i] - EC_reel[i]) / EC_reel[i] * 100, 2) # Relative error in %
        ]
        for i in range(len(EC_test))
    ]
]

# Print table using tabulate for better formatting
print(tabulate(table, headers="firstrow", tablefmt="grid", colalign=("center", "center")))


In [ ]:
""" 
Energy Consumption are calculated based on the parameters collected for the different case studies. 
Results are compared with the empirical EC results realized by car manufacturer.
The table produced by this algorithm presents the results for BEV.
It displays the car model, engine and driving cycle of the test, 
then compares the theoretical EC calculation with with Empirical Results, and measures the relative error of the model.

"""

EC_test = []
EC_reel = []
M_list = []
D_list = []

# Iterate through the testEV list and calculate EC values
for k in range(len(testEV_list[1])):
    try:
        body_name, cycle_name, engine_name, trans_name = testEV_list[1][k][:4]
        EC_reel.append(float(testEV_list[1][k][4]))  # Convert EC_reel to float
        EC_test.append(EC_calculation_elec(bodyEV_dict[body_name], engineEV_dict[engine_name], cycleEV_dict[cycle_name], transEV_dict[trans_name])[0])
    except KeyError as e:
        print(f"Warning: Missing key {e} in one of the dictionaries. Skipping row {k}.")
    except ValueError:
        print(f"Warning: Invalid EC value in row {k}. Skipping.")

# Convert lists to NumPy arrays
EC_test = np.array(EC_test)
EC_reel = np.array(EC_reel, dtype=float)

# Prepare table for display
table = [
    ['Car', 'Cycle', "Th. EC[l/100km]", "Emp. EC[l/100km]", "Error", "Rel.Error[%]"],
    *[
        [
            testEV_list[1][i][0],  # Car name
            testEV_list[1][i][1],  # Cycle name
            round(EC_test[i], 3),  # Test EC
            round(EC_reel[i], 3),  # Reel EC min
            round(EC_test[i] - EC_reel[i], 3),  # Absolute error
            round((EC_test[i] - EC_reel[i]) / EC_reel[i] * 100, 2) if EC_reel[i] != 0 else "N/A"  # Relative error (%)
        ]
        for i in range(len(EC_test))
    ]
]

# Print table using tabulate for better formatting
print(tabulate(table, headers="firstrow", tablefmt="grid", colalign=("center", "center")))


In [ ]:
"""
Graphical representation of the relative error of the model for both GV and BEV.
Three graphs representing the error, classifying the test resp. by car size, by engine and by driving cycle.
"""

# Initialize lists for theoretical and real values
EC_test, EC_reel, EC_test_EV, EC_reel_EV = [], [], [], []
M_list, M_list_EV = [], []

# Compute theoretical and real energy consumption for ICE vehicles
for k in range(len(test_list[1])):
    body_name, cycle_name, engine_name, trans_name = test_list[1][k][:4]
    EC_reel.append(float(test_list[1][k][4]))  # Convert real EC to float
    EC_test.append(EC_calculation_th(body_dict[body_name], engine_dict[engine_name], cycle_dict[cycle_name], trans_dict[trans_name])[0])
    M_list.append(body_dict[body_name][0])  # Extract vehicle mass

# Compute theoretical and real energy consumption for EVs
for k in range(len(testEV_list[1])):
    bodyEV_name, cycleEV_name, engineEV_name, transEV_name = testEV_list[1][k][:4]
    EC_reel_EV.append(float(testEV_list[1][k][4]))  # Convert real EC to float
    EC_test_EV.append(EC_calculation_elec(bodyEV_dict[bodyEV_name], engineEV_dict[engineEV_name], cycleEV_dict[cycleEV_name], transEV_dict[transEV_name])[0])
    M_list_EV.append(bodyEV_dict[bodyEV_name][0])  # Extract vehicle mass

# Convert lists to numpy arrays
EC_test = np.array(EC_test)
EC_reel = np.array(EC_reel)
EC_test_EV = np.array(EC_test_EV)
EC_reel_EV = np.array(EC_reel_EV)

# Calculate relative errors
errors_rel = (EC_test - EC_reel) / EC_reel * 100
errors_rel_EV = (EC_test_EV - EC_reel_EV) / EC_reel_EV * 100
errors_all = np.concatenate((errors_rel, errors_rel_EV))

# Extract cycle and engine names
cycles = [test_list[1][i][1] for i in range(len(test_list[1]))]
engine = [test_list[1][i][2] for i in range(len(test_list[1]))]
cycles_EV = [testEV_list[1][i][5] for i in range(len(testEV_list[1]))]
engine_EV = [testEV_list[1][i][2] for i in range(len(testEV_list[1]))]

cycles_all = cycles + cycles_EV
engine_all = engine + engine_EV

# Define vehicle weight categories
Group = np.array(['Mini', 'Small', 'Medium', 'Large', 'SUV', 'Van'])
M_limit = [1000, 1300, 1450, 1650, 1900]

grouping = Group[np.searchsorted(M_limit, M_list)]
grouping_EV = Group[np.searchsorted(M_limit, M_list_EV)]
grouping_all = np.concatenate((grouping, grouping_EV))

# Determine max error for scaling the plots
max_errors = np.max(np.abs(errors_all)) * 1.2  # Add 20% margin

# Initialize plots
fig, axs = plt.subplots(1, 3, figsize=(20, 10))

for ax in axs:
    ax.set_xlim(-max_errors, max_errors)
    ax.axvline(x=0, color='grey', linestyle='--')

# -------- 1. Errors by Weight Category --------
axs[0].scatter(errors_rel, grouping, color='green', marker='o', zorder=5)
axs[0].scatter(errors_rel_EV, grouping_EV, color='purple', marker='x', zorder=5)

for cat in np.unique(grouping_all):
    indices = np.where(grouping_all == cat)[0]
    min_val, max_val = np.min(errors_all[indices]), np.max(errors_all[indices])
    axs[0].barh(cat, max_val - min_val, left=min_val, color='lightgreen')

axs[0].set_xlabel('Relative Error [%]', fontsize=16)
axs[0].set_ylabel('Category Name', fontsize=16)
axs[0].set_title('Error by Weight Category', fontsize=16)
axs[0].tick_params(labelsize=14)
axs[0].grid(axis='y')

# -------- 2. Errors by Engine Type --------
axs[1].scatter(errors_rel, engine, color='blue', marker='o', zorder=5)
axs[1].scatter(errors_rel_EV, engine_EV, color='purple', marker='x', zorder=5)

for eng in np.unique(engine_all):
    indices = np.where(np.array(engine_all) == eng)[0]
    min_val, max_val = np.min(errors_all[indices]), np.max(errors_all[indices])
    axs[1].barh(eng, max_val - min_val, left=min_val, color='lightskyblue')

axs[1].set_xlabel('Relative Error [%]', fontsize=16)
axs[1].set_title('Error by Engine Type', fontsize=16)
axs[1].tick_params(labelsize=14)
axs[1].grid(axis='y')

# -------- 3. Errors by Driving Cycle --------
axs[2].scatter(errors_rel, cycles, color='red', marker='o', zorder=5)
axs[2].scatter(errors_rel_EV, cycles_EV, color='purple', marker='x', zorder=5)

for cyc in np.unique(cycles_all):
    indices = np.where(np.array(cycles_all) == cyc)[0]
    min_val, max_val = np.min(errors_all[indices]), np.max(errors_all[indices])
    axs[2].barh(cyc, max_val - min_val, left=min_val, color='lightcoral')

axs[2].set_xlabel('Relative Error [%]', fontsize=16)
axs[2].set_title('Error by Driving Cycle', fontsize=16)
axs[2].tick_params(labelsize=14)
axs[2].grid(axis='y')

# Show plots
plt.tight_layout()
plt.show()
